# Highway2Vec Embedder

In [ ]:
import geopandas as gpd
from IPython.display import display

## Get an area to embed

In [ ]:
from srai.utils import geocode_to_region_gdf

area_gdf = geocode_to_region_gdf("Wrocław, PL")
area_gdf.plot()

## Regionize the area with a regionizer

In [ ]:
from srai.regionizers import H3Regionizer

regionizer = H3Regionizer(9)
regions_gdf = regionizer.transform(area_gdf)
print(len(regions_gdf))
display(regions_gdf.head(3))
regions_gdf.plot()

## Download a road infrastructure for the area

In [ ]:
from srai.loaders import OSMWayLoader
from srai.loaders.osm_way_loader import NetworkType

loader = OSMWayLoader(NetworkType.DRIVE)
nodes_gdf, edges_gdf = loader.load(area_gdf)

display(nodes_gdf.head(3))
display(edges_gdf.head(3))

ax = edges_gdf.plot(linewidth=1, figsize=(12, 7))
nodes_gdf.plot(ax=ax, markersize=3, color="red")

## Find out which edges correspond to which regions 

In [ ]:
from srai.joiners import IntersectionJoiner

joiner = IntersectionJoiner()
joint_gdf = joiner.transform(regions_gdf, edges_gdf)
joint_gdf

## Get the embeddings for regions based on the road infrastructure

In [ ]:
from srai.embedders import Highway2VecEmbedder
from pytorch_lightning import seed_everything

seed_everything(42)

embedder = Highway2VecEmbedder()
embeddings = embedder.fit_transform(regions_gdf, edges_gdf, joint_gdf)
embeddings